In [2]:
import requests
import time
import pandas as pd

# ==============================
# 🔑 CONFIGURATION
# ==============================

API_KEY = "RGAPI-10b59e68-450c-45dd-bfe3-ecd0dca26a92"  # <-- PASTE YOUR API KEY
REGION = "sea"  # use sea since it worked for you

headers = {
    "X-Riot-Token": API_KEY
}

# ==============================
# 📋 MATCH IDS
# ==============================

match_ids = [
    "SG2_133897261",
    "SG2_133864236",
    "SG2_133849997",
    "SG2_133352026",
    "SG2_133178865",
    "SG2_133159578",
    "SG2_132963562",
    "SG2_132961475",
    "SG2_132952628",
    "SG2_132813731",
    "SG2_132759588",
    "SG2_132756618",
    "SG2_132756213",
    "SG2_132746271",
    "SG2_132737891",
    "SG2_132713419",
    "SG2_132590067",
    "SG2_132571004",
    "SG2_132562763",
    "SG2_132527823",
    "SG2_132522590",
    "SG2_132335641",
    "SG2_132332021",
    "SG2_132321042",
    "SG2_132314755",
    "SG2_132134098",
    "SG2_132125912",
    "SG2_132121734",
    "SG2_132088337",
    "SG2_132084685",
    "SG2_132081492",
    "SG2_132066514",
    "SG2_131664015",
    "SG2_131635861",
    "SG2_131633450",
    "SG2_131628578",
    "SG2_131624663",
    "SG2_131618283",
    "SG2_131387384",
    "SG2_131321320",
    "SG2_131308366",
    "SG2_131167785",
    "SG2_131168396",
    "SG2_131120063",
    "SG2_131090885",
    "SG2_131076888",
    "SG2_131068225",
    "SG2_130943213",
    "SG2_130914172",
    "SG2_130896177",
    "SG2_130715862",
    "SG2_130713600",
    "SG2_130709493",
    "SG2_130706710",
    "SG2_130696340",
    "SG2_130683085",
    "SG2_130625155",
    "SG2_130460244",
    "SG2_130458746",
    "SG2_130459181",
    "SG2_130451909",
    "SG2_130451086",
    "SG2_130439905",
    "SG2_130427430",
    "SG2_130374103",
    "SG2_130337686",
    "SG2_130329455",
    "SG2_130320760",
    "SG2_130065081",
    "SG2_129972925",
    "SG2_129960450",
    "SG2_129956464",
    "SG2_129953556",
    "SG2_129954010",
    "SG2_129950329",
    "SG2_129934373",
    "SG2_129919026",
    "SG2_129796173",
    "SG2_129789772",
    "SG2_129783848",
    "SG2_129753588",
    "SG2_129374134",
    "SG2_129238327",
    "SG2_129229846",
    "SG2_129181810",
    "SG2_128751950",
    "SG2_128750310",
    "SG2_128739690",
    "SG2_128732780",
    "SG2_128728921",
    "SG2_128555522",
    "SG2_128471745",
    "SG2_128469503",
    "SG2_128458433",
    "SG2_128457226",
    "SG2_128453386",
    "SG2_128422860",
    "SG2_128415654",
    "SG2_128207980",
    "SG2_128196943"
]

# ==============================
# 🚀 FETCH MATCH DATA
# ==============================

all_rows = []

for match_id in match_ids:
    url = f"https://{REGION}.api.riotgames.com/tft/match/v1/matches/{match_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        match_data = response.json()
        game_info = match_data["info"]

        # Game-level info
        game_meta = {
            "match_id": match_id,
            "endOfGameResult": game_info.get("end_of_game_result"),
            "gameCreation": game_info.get("game_datetime"),
            "gameId": match_data.get("metadata", {}).get("match_id"),
            "game_datetime": game_info.get("game_datetime"),
            "game_length": game_info.get("game_length"),
            "game_version": game_info.get("game_version"),
            "game_variation": game_info.get("game_variation"),
            "mapId": game_info.get("map_id"),
            "queue_id": game_info.get("queue_id"),
            "queueId": game_info.get("queue_id"),  # deprecated
            "tft_game_type": game_info.get("tft_game_type"),
            "tft_set_core_name": game_info.get("tft_set_core_name"),
            "tft_set_number": game_info.get("tft_set_number")
        }

        # Participant-level info
        for participant in game_info.get("participants", []):
            row = game_meta.copy()
            row.update({
                "placement": participant.get("placement"),
                "level": participant.get("level"),
                "gold_left": participant.get("gold_left"),
                "last_round": participant.get("last_round"),
                "players_eliminated": participant.get("players_eliminated"),
                "time_eliminated": participant.get("time_eliminated"),
                "total_damage_to_players": participant.get("total_damage_to_players"),
                "win": participant.get("win"),
                "companion": str(participant.get("companion")),
                "traits": "; ".join(
                    [f"{t['name']}({t['num_units']})" for t in participant.get("traits", [])]
                ),
                "units": "; ".join(
                    [f"{u.get('character_id')}[{','.join(map(str, u.get('items', [])))}]" for u in participant.get("units", [])]
                )
            })

            all_rows.append(row)

        print(f"✅ Fetched {match_id}")

    else:
        print(f"❌ Error {response.status_code} for {match_id}")

    # Rate limit protection
    time.sleep(1.2)

# ==============================
# 💾 SAVE TO CSV
# ==============================

df = pd.DataFrame(all_rows)
df.to_csv("tft_dataset.csv", index=False)

print("🎉 DONE! Dataset saved as tft_dataset.csv")

✅ Fetched SG2_133897261
✅ Fetched SG2_133864236
✅ Fetched SG2_133849997
✅ Fetched SG2_133352026
✅ Fetched SG2_133178865
✅ Fetched SG2_133159578
✅ Fetched SG2_132963562
✅ Fetched SG2_132961475
✅ Fetched SG2_132952628
✅ Fetched SG2_132813731
✅ Fetched SG2_132759588
✅ Fetched SG2_132756618
✅ Fetched SG2_132756213
✅ Fetched SG2_132746271
✅ Fetched SG2_132737891
✅ Fetched SG2_132713419
✅ Fetched SG2_132590067
✅ Fetched SG2_132571004
✅ Fetched SG2_132562763
✅ Fetched SG2_132527823
✅ Fetched SG2_132522590
✅ Fetched SG2_132335641
✅ Fetched SG2_132332021
✅ Fetched SG2_132321042
✅ Fetched SG2_132314755
✅ Fetched SG2_132134098
✅ Fetched SG2_132125912
✅ Fetched SG2_132121734
✅ Fetched SG2_132088337
✅ Fetched SG2_132084685
✅ Fetched SG2_132081492
✅ Fetched SG2_132066514
✅ Fetched SG2_131664015
✅ Fetched SG2_131635861
✅ Fetched SG2_131633450
✅ Fetched SG2_131628578
✅ Fetched SG2_131624663
✅ Fetched SG2_131618283
✅ Fetched SG2_131387384
✅ Fetched SG2_131321320
✅ Fetched SG2_131308366
✅ Fetched SG2_13